# B2

In [ ]:
import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist
import torch
import arviz
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
def func_to_minimize(x):
    return torch.sin(20 * x) + 2 * torch.cos(14 * x) - 2 * torch.sin(6 * x)

In [ ]:
# (x_data, y_data) correspond to D and will be updated gradually
# we initialize them to the points from B1.
pyro.clear_param_store()
pyro.set_rng_seed(1351)
x_data = torch.tensor([-1, -0.5, 0, 0.5, 1])
y_data = torch.sin(20 * x_data) + 2 * torch.cos(14 * x_data) - 2 * torch.sin(6 * x_data)
rbm_kernel = gp.kernels.RBF(input_dim = 1)
rbm_kernel.variance = pyro.nn.PyroSample(dist.LogNormal(0, 2))
rbm_kernel.lengthscale = pyro.nn.PyroSample(dist.LogNormal(-1, 1))
n_x_new = 200
x_new = torch.linspace(-1, 1, steps = n_x_new)
n_chains = 1
n_warmup = 2000

def get_param_sample(x_data, y_data):
    pyro.clear_param_store()
    gpr = gp.models.GPRegression(x_data, y_data, rbm_kernel, noise=torch.tensor(1e-4))
    nuts_kernel = pyro.infer.mcmc.NUTS(gpr.model)
    mcmc = pyro.infer.mcmc.MCMC(nuts_kernel, warmup_steps = n_warmup, num_samples = 1, num_chains = 1)
    mcmc.run()
    samples = mcmc.get_samples()
    return samples["kernel.lengthscale"], samples["kernel.variance"]

def get_param_samples(x_data, y_data, n_samples_per_chain, n_warmup = n_warmup):
    pyro.clear_param_store()
    gpr = gp.models.GPRegression(x_data, y_data, rbm_kernel, noise=torch.tensor(1e-4))
    nuts_kernel = pyro.infer.mcmc.NUTS(gpr.model)
    mcmc = pyro.infer.mcmc.MCMC(nuts_kernel, warmup_steps = n_warmup, num_samples = n_samples_per_chain, num_chains = n_chains)
    mcmc.run()
    samples = mcmc.get_samples()
    return samples["kernel.lengthscale"], samples["kernel.variance"]

In [ ]:
n_iter = 15
l = 1.5 # lambda for 
## We also count how many points we add, since we skip an iteration if we get a non positive-definite covariance matrix
count_added = 0
m_tensor = torch.empty(n_iter, n_x_new)
v_tensor = torch.empty(n_iter, n_x_new)
acq_fun_tensor = torch.empty(n_iter, n_x_new)
n_samples_per_chain = 20000
n_param_samples = n_samples_per_chain * n_chains
f_samples = torch.empty(n_param_samples, n_x_new)
count_added = 0
f_tensor = torch.empty(n_iter, n_x_new)
m_tensor = torch.empty(n_iter, n_x_new)
v_tensor = torch.empty(n_iter, n_x_new)

for i in range(n_iter):
    # Sample a lot for m and v
    pyro.clear_param_store()
    lengthscale_samples, variance_samples = get_param_samples(x_data, y_data, n_samples_per_chain)
    m_sample = torch.zeros(n_x_new)
    v_sample = torch.zeros(n_x_new)
    for j in range(n_param_samples):
        pyro.clear_param_store()
        rbm_kernel_sample = gp.kernels.RBF(input_dim = 1,
                                       variance = variance_samples[j],
                                       lengthscale = lengthscale_samples[j])
        gpr_sample = gp.models.GPRegression(x_data, y_data, rbm_kernel_sample,
                                            noise=torch.tensor(1e-4))
        m_sample, v_sample = gpr_sample(x_new, full_cov = False, noiseless = True)
        f_dist = dist.MultivariateNormal(m_sample, covariance_matrix=torch.diag(v_sample))
        f_sample = f_dist.sample()
        f_samples[j, :] = f_sample
    m_post = f_samples.mean(dim = 0)
    v_post = f_samples.var(dim = 0)
    
    # Sample a single f*
    pyro.clear_param_store()
    lengthscale_sample, variance_sample = lengthscale_samples[0], variance_samples[0]
    rbm_kernel_sample = gp.kernels.RBF(input_dim = 1,
                                       variance = variance_sample,
                                       lengthscale = lengthscale_sample)
    gpr_sample = gp.models.GPRegression(x_data, y_data, rbm_kernel_sample,
                                        noise=torch.tensor(1e-4))
    m_sample, cov_sample = gpr_sample(x_new, full_cov = True, noiseless = False)

    try:
        f_dist = dist.MultivariateNormal(m_sample, covariance_matrix=cov_sample)
        f_sample = f_dist.sample()
        x_data = torch.cat((x_data, torch.tensor([x_new[torch.argmin(f_sample)]])))
        y_data = torch.cat((y_data, torch.tensor([func_to_minimize(x_data[-1])])))
        m_tensor[count_added, :] = m_post
        v_tensor[count_added, :] = v_post
        f_tensor[count_added, :] = f_sample
        count_added += 1
    except ValueError:
        print("Not invertible cov")
    print(x_data)

In [ ]:
def plot_step(i, filename = None):
    n_start_points = 5 # Nr of data points used to begin with
    # Extract values from tensors
    m = m_tensor[i, :].detach().numpy()
    v = v_tensor[i, :].detach().numpy()
    f_star = f_tensor[i, :].detach().numpy()
    xs = x_data[0:(n_start_points + i)].detach().numpy()
    ys = y_data[0:(n_start_points + i)].detach().numpy()
    x_added = x_data[n_start_points + i]
    y_added = y_data[n_start_points + i]
    x_new_np = x_new.detach().numpy()
    f_values = np.sin(20 * x_new_np) + 2 * np.cos(14 * x_new_np) - 2 * np.sin(6 * x_new_np)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x_new_np, m, c='b')
    plt.fill_between(x_new_np,
                     m - 2 * np.sqrt(v),
                     m + 2 * np.sqrt(v),
                     alpha=0.2)
    ax.plot(x_new_np, f_values, c='g')
    ax.plot(x_new_np, f_star, c='purple')
    ax.scatter(xs, ys, c='r', marker='o')
    ax.scatter(x_added, y_added, c='black', marker='o')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.legend(['m(x)', '+/- 2*sqrt(v(x))', 'true f', 'sampled f*', 'known f(x)', 'new point'],
              loc = "upper left", ncol = 3)
    plt.title("Iteration " + str(i))
    plt.show()
    if filename:
        fig.savefig('figures/' + filename, format='png', dpi=1200)

In [ ]:
plot_step(0, 'b2_iter0.png')

In [ ]:
plot_step(5, 'b2_iter5.png')

In [ ]:
plot_step(10, 'b2_iter10.png')

Spørgsmål: Skal m og v bare tages fra den ene sample af theta (ligesom f*) eller skal man tage flere samples af theta for at få et ordentligt estimat af m og v?

In [ ]:
torch.save(f_tensor, "results/f_tensor.pt")
torch.save(m_tensor, "results/m_tensor.pt")
torch.save(v_tensor, "results/v_tensor.pt")
torch.save(x_data, "results/x_data.pt")
torch.save(y_data, "results/y_data.pt")

Observations om kørsler:

- Den finder ikke altid det globale minimum, da den kan blive fanget i det lokale minimum omkring x = -0.7. Selv når den er fanget omkring -0.7 kan man dog være heldig at den hopper over i nærheden af det rigtige minimum.
- Den bruger nogen gange flere iterationer på at tilføje -1 som ekstra punkt i starten, fordi det er relativt sandsynligt at sample en f* som har minimum i -1 (fordi det er det mindste af datapunkterne vi har). Dette er bare spild af iterationer, da det slet ikke tilføjer ny information.
- En gang imellem får den en ikke invertibel kovarians (selvom vi sampler med støj og tilføjer jitter). Vi springer bare den iteration over - man kunne også evt. skrue op for støj eller jitter der lægges til diagonalen i kovariansmatricen.


I praksis ville man også have brug for en metode til at beslutte hvornår man stopper kørslen og tager det sidste punkt som sit estimat.

## Alternativ metode: UCB (upper confidence bound)

In [ ]:
# (x_data, y_data) correspond to D and will be updated gradually
# we initialize them to the points from B1.
pyro.clear_param_store()
pyro.set_rng_seed(1357)
x_data = torch.tensor([-1, -0.5, 0, 0.5, 1])
y_data = torch.sin(20 * x_data) + 2 * torch.cos(14 * x_data) - 2 * torch.sin(6 * x_data)
rbm_kernel = gp.kernels.RBF(input_dim = 1)
rbm_kernel.variance = pyro.nn.PyroSample(dist.LogNormal(0, 2))
rbm_kernel.lengthscale = pyro.nn.PyroSample(dist.LogNormal(-1, 1))
n_x_new = 200
x_new = torch.linspace(-1, 1, steps = n_x_new)
n_chains = 1

def get_param_samples(x_data, y_data, n_samples_per_chain, n_warmup = 2000):
    pyro.clear_param_store()
    gpr = gp.models.GPRegression(x_data, y_data, rbm_kernel, noise=torch.tensor(1e-4))
    nuts_kernel = pyro.infer.mcmc.NUTS(gpr.model)
    mcmc = pyro.infer.mcmc.MCMC(nuts_kernel, warmup_steps = n_warmup, num_samples = n_samples_per_chain, num_chains = n_chains)
    mcmc.run()
    samples = mcmc.get_samples()
    return samples["kernel.lengthscale"], samples["kernel.variance"]

In [ ]:
n_iter = 15
l = 1.5 # lambda for 
## We also count how many points we add, since we skip an iteration if we get a non positive-definite covariance matrix
count_added = 0
m_tensor = torch.empty(n_iter, n_x_new)
v_tensor = torch.empty(n_iter, n_x_new)
acq_fun_tensor = torch.empty(n_iter, n_x_new)
n_samples_per_chain = 20000
n_param_samples = n_samples_per_chain * n_chains
f_samples = torch.empty(n_param_samples, n_x_new)
for i in range(n_iter):
    pyro.clear_param_store()
    lengthscale_samples, variance_samples = get_param_samples(x_data, y_data, n_samples_per_chain)
    m_sample = torch.zeros(n_x_new)
    v_sample = torch.zeros(n_x_new)
    for j in range(n_param_samples):
        pyro.clear_param_store()
        rbm_kernel_sample = gp.kernels.RBF(input_dim = 1,
                                       variance = variance_samples[j],
                                       lengthscale = lengthscale_samples[j])
        gpr_sample = gp.models.GPRegression(x_data, y_data, rbm_kernel_sample,
                                            noise=torch.tensor(1e-4))
        m_sample, v_sample = gpr_sample(x_new, full_cov = False, noiseless = True)
        f_dist = dist.MultivariateNormal(m_sample, covariance_matrix=torch.diag(v_sample))
        f_sample = f_dist.sample()
        f_samples[j, :] = f_sample
    ## Catch error with non positive-definite covariance matrix to avoid stopping the loop
    m_post = f_samples.mean(dim = 0)
    v_post = f_samples.var(dim = 0)
    try:
        acq_fun = m_post - l * torch.sqrt(v_post)
        x_data = torch.cat((x_data, torch.tensor([x_new[torch.argmin(acq_fun)]])))
        y_data = torch.cat((y_data, torch.tensor([func_to_minimize(x_data[-1])])))
        m_tensor[count_added, :] = m_post
        v_tensor[count_added, :] = v_post
        acq_fun_tensor[count_added, :] = acq_fun
        count_added += 1
    except ValueError:
        print("Not invertible cov")
    print(x_data)

In [ ]:
def plot_step(i, filename = None):
    n_start_points = 5 # Nr of data points used to begin with
    # Extract values from tensors
    m = m_tensor[i, :].detach().numpy()
    v = v_tensor[i, :].detach().numpy()
    acq_fun = acq_fun_tensor[i, :].detach().numpy()
    xs = x_data[0:(n_start_points + i)].detach().numpy()
    ys = y_data[0:(n_start_points + i)].detach().numpy()
    x_added = x_data[n_start_points + i]
    y_added = y_data[n_start_points + i]
    x_new_np = x_new.detach().numpy()
    f_values = np.sin(20 * x_new_np) + 2 * np.cos(14 * x_new_np) - 2 * np.sin(6 * x_new_np)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x_new_np, m, c='b')
    plt.fill_between(x_new_np,
                     m - 2 * np.sqrt(v),
                     m + 2 * np.sqrt(v),
                     alpha=0.2)
    ax.plot(x_new_np, f_values, c='g')
    ax.plot(x_new_np, acq_fun, c='purple')
    ax.scatter(xs, ys, c='r', marker='o')
    ax.scatter(x_added, y_added, c='black', marker='o')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.legend(['m(x)', '+/- 2*sqrt(v(x))', 'true f', 'acq. function', 'known f(x)', 'new point'],
               loc='upper left', ncol = 3)
    plt.title("Iteration " + str(i))
    plt.show()
    if filename:
        fig.savefig('figures/' + filename, format='png', dpi=1200)    

In [ ]:
plot_step(0, 'b2_ucb_iter0.png')

In [ ]:
plot_step(5, 'b2_ucb_iter5.png')

In [ ]:
plot_step(10, 'b2_ucb_iter10.png')

In [ ]:
# torch.save(f_tensor, "results_step/acq_fun_tensor.pt")
torch.save(m_tensor, "results_step/m_tensor.pt")
torch.save(v_tensor, "results_step/v_tensor.pt")
torch.save(x_data, "results_step/x_data.pt")
torch.save(y_data, "results_step/y_data.pt")